In [46]:
import requests as r
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
token = os.getenv("COINAPI2")
from datetime import datetime, timedelta
from processor.processor import Processor as p
from tqdm import tqdm

In [2]:
token

'DC8B28EB-89AB-4FE8-9FD6-302E5C575BA5'

In [4]:
## Getting Symbols
url = 'https://rest.coinapi.io/v1/symbols'
headers = {'X-CoinAPI-Key' : token}
params =  {'symbol_type':'SPOT',
          'asset_id_quote':'USD'}
response = r.get(url, headers=headers,params=params)
response.json()

[{'symbol_id': 'BEQUANT_SPOT_BTC_CHF',
  'exchange_id': 'BEQUANT',
  'symbol_type': 'SPOT',
  'asset_id_base': 'BTC',
  'asset_id_quote': 'CHF',
  'data_end': '2022-01-06',
  'data_quote_end': '2022-01-06T15:30:11.9892184Z',
  'price': 39487.8,
  'symbol_id_exchange': 'BTCCHF',
  'asset_id_base_exchange': 'BTC',
  'asset_id_quote_exchange': 'CHF',
  'price_precision': 0.1,
  'size_precision': 1e-06},
 {'symbol_id': 'OKEX_OPT_ETH_USD_211205_3800_C',
  'exchange_id': 'OKEX',
  'symbol_type': 'OPTION',
  'asset_id_base': 'ETH',
  'asset_id_quote': 'USD',
  'option_type_is_call': True,
  'option_strike_price': 3800.0,
  'option_contract_unit': 1.0,
  'option_exercise_style': 'EUROPEAN',
  'option_expiration_time': '2021-12-05T08:00:00.0000000Z',
  'data_end': '2021-12-05',
  'data_quote_end': '2021-12-05T07:59:00.1190000Z',
  'symbol_id_exchange': 'ETH_USD_211205_3800_C',
  'asset_id_base_exchange': 'ETH',
  'asset_id_quote_exchange': 'USD',
  'price_precision': 0.0001,
  'size_precision':

In [5]:
symbols = pd.DataFrame(response.json())
symbols.head()

,symbol_id,exchange_id,symbol_type,asset_id_base,asset_id_quote,data_end,data_quote_end,price,symbol_id_exchange,asset_id_base_exchange,...,volume_1day,volume_1day_usd,index_id,index_display_name,index_display_description,contract_delivery_time,contract_unit,contract_id,contract_display_name,contract_display_description
0,BEQUANT_SPOT_BTC_CHF,BEQUANT,SPOT,BTC,CHF,2022-01-06,2022-01-06T15:30:11.9892184Z,39487.8,BTCCHF,BTC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OKEX_OPT_ETH_USD_211205_3800_C,OKEX,OPTION,ETH,USD,2021-12-05,2021-12-05T07:59:00.1190000Z,NaN,ETH_USD_211205_3800_C,ETH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OKEX_OPT_ETH_USD_211205_3000_C,OKEX,OPTION,ETH,USD,2021-12-05,2021-12-05T07:59:00.0340000Z,NaN,ETH_USD_211205_3000_C,ETH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OKEX_OPT_ETH_USD_211205_3200_C,OKEX,OPTION,ETH,USD,2021-12-05,2021-12-05T07:58:57.4110000Z,NaN,ETH_USD_211205_3200_C,ETH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OKEX_OPT_ETH_USD_211205_3550_C,OKEX,OPTION,ETH,USD,2021-12-05,2021-12-05T07:58:26.4970000Z,NaN,ETH_USD_211205_3550_C,ETH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
spots_usd = symbols[(symbols["asset_id_quote"]=="USD") & (symbols["symbol_type"]=="SPOT") & (symbols["exchange_id"]=="COINBASE")]

In [7]:
spots_usd.to_csv("symbols.csv")

In [8]:
coinbase_symbols = spots_usd["symbol_id"].unique()

In [9]:
coinbase_symbols[:2]

array(['COINBASE_SPOT_BTC_USD', 'COINBASE_SPOT_ETH_USD'], dtype=object)

In [10]:
start = datetime(2021,1,1).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
examples = []
for symbol in coinbase_symbols[:2]:
    try:
        url = f'https://rest.coinapi.io/v1/ohlcv/{symbol}/history?period_id=1MIN'
        headers = {'X-CoinAPI-Key' : token}
        params = {'time_start':start,
                     'time_end':end,
                 'limit':100000}
        print(url)
        response = r.get(url, headers=headers,params=params)
        data = pd.DataFrame(response.json())
        data['symbol'] = symbol.split("_")[2]
        examples.append(data)
    except Exception as e:
        print(symbol,str(e))

https://rest.coinapi.io/v1/ohlcv/COINBASE_SPOT_BTC_USD/history?period_id=1MIN
https://rest.coinapi.io/v1/ohlcv/COINBASE_SPOT_ETH_USD/history?period_id=1MIN


In [11]:
timeline = pd.concat(examples)

In [12]:
test_set = timeline.pivot_table(columns="symbol",index="time_close",values="price_close").reset_index()
test_set.rename(columns={"time_close":"date"},inplace=True)
test_set = p.column_date_processing(test_set)

In [83]:
final = test_set.sort_values("date").fillna(method="ffill").iloc[1:].reset_index(drop=True)

In [14]:
final.to_csv("crypto_test.csv")

In [84]:
final.head(1)

symbol,date,btc,eth
0,2021-01-01 00:00:59.977129,29026.97,738.9


In [112]:
futures[futures["signal"] < -0.05]

symbol,date,btc,eth,signal
146663,2021-02-20 22:16:59.950730,56033.40,1936.00,-0.050399
146667,2021-02-20 22:18:59.933223,55949.00,1931.37,-0.052670
146673,2021-02-20 22:21:59.957000,56061.94,1931.97,-0.052375
146674,2021-02-20 22:22:59.648181,56061.94,1935.38,-0.050703
146675,2021-02-20 22:22:59.855941,56099.36,1935.38,-0.050703
...,...,...,...,...
199994,2021-03-11 10:42:58.686812,55168.19,1753.06,-0.140130
199995,2021-03-11 10:42:59.101644,55168.19,1755.44,-0.138963
199996,2021-03-11 10:43:55.651832,55224.00,1755.44,-0.138963
199997,2021-03-11 10:44:59.910494,55110.93,1755.44,-0.138963


In [ ]:
start = final.iloc[0]["date"]
end = final.iloc[-1]["date"]
trades = []
for symbol in tqdm(["btc","eth"]):
    for retrack_days in tqdm(range(1,8,1)):
        for s in tqdm(range(0,55,5)):
            for r in range(5,55,5):
                signal = float(s/100)
                req = float(r/100)
                date = start
                trackers = {"btc":final.iloc[0]["btc"].item()
                           ,"eth":final.iloc[0]["eth"].item()}
                while date < end:
                    try:
                        initial = trackers[symbol]
                        futures = final[final["date"]>date]
                        futures["signal"] = (futures[symbol] - initial) / initial
                        futures.sort_values("date",inplace=True)
                        entries = futures[(futures["signal"] <= -signal) & (futures["date"] <= date+timedelta(days=retrack_days))]
                        if entries.index.size < 1:
                            if (end-date).days < retrack_days:
                                break
                            else:
                                trackers[symbol] = futures[(futures["date"] >= date+timedelta(days=retrack_days))].iloc[0][symbol]
                                date = futures[(futures["date"] >= date+timedelta(days=retrack_days))].iloc[0]["date"]
                        else:
                            trade = entries.iloc[0]
                            exits = final[final["date"]>trade["date"]]
                            bp = trade[symbol]
                            exits["delta"] = (exits[symbol] - bp) / bp
                            profits = exits[exits["delta"] >= req]
                            if profits.index.size < 1:
                                breakeven = exits[exits["delta"]>=0]
                                if breakeven.index.size < 1:
                                    exit = exits.iloc[-1]
                                else:
                                    exit = breakeven.iloc[0]
                            else:
                                exit = profits.iloc[0]
                            trade["sell_date"] = exit["date"]
                            trade["sell_price"] = exit[symbol]
                            trade["buy_price"] = bp
                            trade["delta"] = (trade["sell_price"] - trade["buy_price"])/ trade["buy_price"]
                            trade["signal"] = signal
                            trade["req"] = req
                            trade["symbol"] = symbol
                            trade["retrack_days"] = retrack_days
                            trades.append(trade)
                            trackers[symbol] = exit[symbol].item()
                            date = exit["date"] + timedelta(minutes=1)
                    except Exception as e:
                        print(date,str(e))
                        date = date + timedelta(minutes=1)

  0%|                                                                                                                                 | 0/7 [00:00<?, ?it/s]

  0%|                                                                                                                                | 0/10 [00:00<?, ?it/s]/var/folders/b_/gmd7ll511zj4c6z0mlqhgln00000gn/T/ipykernel_1886/1976567502.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  futures["signal"] = (futures[symbol] - initial) / initial
/Volumes/WIN10/projects/longshot_venv/lib/python3.10/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/p

 43%|███████████████████████████████████████████████████▊                                                                     | 3/7 [04:44<05:31, 82.81s/it]

  0%|                                                                                                                                | 0/10 [00:00<?, ?it/s]

 10%|████████████                                                                                                            | 1/10 [00:02<00:25,  2.86s/it]

 20%|████████████████████████                                                                                                | 2/10 [00:05<00:21,  2.75s/it]

 30%|████████████████████████████████████                                                                                    | 3/10 [00:08<00:19,  2.79s/it]

 40%|████████████████████████████████████████████████                                                                        | 4/10 [00:11<00:17,  2.88s/it]

 50%|███████████████████████████████████████████████

  0%|                                                                                                                                 | 0/7 [00:00<?, ?it/s]

  0%|                                                                                                                                | 0/10 [00:00<?, ?it/s]

 10%|████████████                                                                                                            | 1/10 [00:06<01:02,  6.95s/it]

 20%|████████████████████████                                                                                                | 2/10 [00:15<01:02,  7.83s/it]

 30%|████████████████████████████████████                                                                                    | 3/10 [00:27<01:09,  9.94s/it]

 40%|████████████████████████████████████████████████                                                                        | 4/10 [00:41<01:07, 11.26s/it]

 50%|███████████████████████████████████████████████

In [ ]:
t = pd.DataFrame(trades)
t.to_csv("test_trades.csv")

In [ ]:
t

In [ ]:
analysis = []
for symbol in tqdm(["btc","eth"]):
    for retrack_days in range(1,7,1):
        for s in tqdm(range(0,45,5)):
            for r in range(5,45,5):
                try:
                    signal = float(s/100)
                    req = float(r/100)
                    iteration_trades = t[(t["symbol"] == symbol) & (t["signal"]==signal) & (t["req"]==req) & (t["retrack_days"]==retrack_days)]
                    initial = 100
                    for delta in iteration_trades["delta"]:
                         initial = initial * (1+delta)
                    iteration_trades["hpr"] = iteration_trades["sell_date"] - iteration_trades["date"]
                    iteration_trades["minutes"] = [x.days * 1440 for x in iteration_trades["hpr"]]
                    minutes = iteration_trades["minutes"].mean()
                    analysis.append({"symbol":symbol,"signal":signal,"req":req,"trades":iteration_trades.index.size,"pv":initial,"minutes":minutes,"retrack_days":retrack_days})
                except Exception as e:
                    print(str(e))

In [ ]:
pd.DataFrame(analysis).sort_values("pv",ascending=False).head(20)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(final["date"],final["btc"])